In [18]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [19]:
%store -r tarjetas
%store -r bancos

In [3]:
tarjetas

Desc Concepto Clearing,Fecha de Pago,Cod Marca Producto,Cod Comercio,Nro Liquidacion,"PERCEPCION IVA R.G. 2408 3,00 %",AJUSTE SIRTAC NOV. 1RA QU,AJUSTE SIRTAC OCT. 2DA QUIN,ARANCEL,CARGO SISTEMA CUOTAS MENS,CARGO LIQUIDACION ELECTR.,...,RETENCION IMP.GANANCIAS,RETENCION ING.BRUTOS,RETENCION ING.BRUTOS SIRTAC,RETENCION IVA,SERVICIO OPER. INTERNAC.,TOTAL DEDUCCIONES,TOTAL LIQUIDACION,TOTAL PAGOS DE COMERCIOS,VENTAS C/DESCUENTO CONTADO,VENTAS C/DTO CUOTAS FINANC. OTORG.
0,2022-11-01,Maestro,4547438,146455,0.00,0.0,0.0,56054.06,0.0,0.0,...,34754.29,137627.00,56055.30,34754.29,0.0,331408.67,6675503.82,0.00,7006912.49,0.00
1,2022-11-01,Master Debit,4547438,146452,1479.55,0.0,0.0,49312.17,0.0,0.0,...,30574.21,121073.88,49313.24,30574.21,0.0,293028.01,5871126.76,37954.57,6164154.77,0.00
2,2022-11-01,Mastercard,4547438,144402,0.00,0.0,0.0,1895.97,0.0,0.0,...,1034.37,1986.00,948.00,3103.12,0.0,9378.88,95954.37,0.00,105333.25,0.00
3,2022-11-01,Mastercard,4547438,145565,2268.06,0.0,0.0,75603.50,0.0,0.0,...,38841.06,74574.83,37802.10,116523.17,0.0,629480.95,3570751.85,0.00,1105457.30,3094775.50
4,2022-11-01,Mastercard,4547438,146367,109.98,0.0,0.0,3665.34,0.0,0.0,...,1692.14,3248.91,1629.05,5076.43,0.0,53166.42,150465.43,0.00,0.00,203631.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790,2022-11-30,Visa Debito,25108439,52976,0.00,0.0,0.0,739.86,0.0,0.0,...,0.00,0.00,739.87,0.00,0.0,1635.10,90848.40,0.00,92483.50,0.00
3791,2022-11-30,Visa Debito,25108471,149351,0.00,0.0,0.0,384.31,0.0,0.0,...,0.00,0.00,384.32,0.00,0.0,849.34,47190.30,0.00,48039.64,0.00
3792,2022-11-30,Visa Debito,25108573,24570,0.00,0.0,0.0,546.22,0.0,0.0,...,0.00,1354.70,546.25,0.00,0.0,2565.70,65715.30,0.00,68281.00,0.00
3793,2022-11-30,Visa Debito,25892842,149353,75.19,0.0,0.0,2505.11,0.0,0.0,...,1553.20,0.00,2505.17,1553.20,0.0,8717.94,304428.06,0.00,313146.00,0.00


In [20]:
tarjeta_visa_debito = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Visa Debito'] #Filtro por la Tarjeta Mastercard Debit
tarjeta_visa_debito = tarjeta_visa_debito.loc[:, (tarjeta_visa_debito!= 0).any(axis=0)] #borro las columnas que unicamente tengan valores 0
tarjeta_visa_debito = tarjeta_visa_debito.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS C/DESCUENTO CONTADO', 'ARANCEL', 'IVA CRED.FISC.COMERCIO S/ARANC 21,00%',
       'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC','PERC.I.B. CORRIENTES R.165/00', 'PERCEPCION IVA R.G. 2408   3,00 %', 
       'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION'])
        #reordeno columnas, aca ya quedan en el orden que es necesario
tarjeta_visa_debito['ARANCEL'] = tarjeta_visa_debito['IVA CRED.FISC.COMERCIO S/ARANC 21,00%']*100/21        
tarjeta_visa_debito.insert(7, 'Iva 21', 
                                tarjeta_visa_debito['IVA CRED.FISC.COMERCIO S/ARANC 21,00%']/tarjeta_visa_debito['ARANCEL'])
                                #agrego columna de control para IVA %21
tarjeta_visa_debito = tarjeta_visa_debito.assign(Diferencia = tarjeta_visa_debito.iloc[:, 5:7].sum(axis=True) 
                    + tarjeta_visa_debito.iloc[:, 8:15].sum(axis=True)
                    - tarjeta_visa_debito['TOTAL DEDUCCIONES']) #agrego columna de control final

In [21]:
sum_columns = [ 'Nro Liquidacion', 'VENTAS C/DESCUENTO CONTADO', 'ARANCEL',
       'IVA CRED.FISC.COMERCIO S/ARANC 21,00%', 'Iva 21', 'RETENCION IVA',
       'PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS',
       'RETENCION ING.BRUTOS', 'RETENCION ING.BRUTOS SIRTAC',
       'PERC.I.B. CORRIENTES R.165/00', 'PERCEPCION IVA R.G. 2408   3,00 %',
       'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}

tarjeta_visa_debito=tarjeta_visa_debito.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [22]:
tarjeta_visa_debito = tarjeta_visa_debito.reset_index()
tarjeta_visa_debito = tarjeta_visa_debito.merge(bancos)

In [23]:
tarjeta_visa_debito

,Cod Marca Producto,Cod Comercio,Nro Liquidacion,VENTAS C/DESCUENTO CONTADO,ARANCEL,"IVA CRED.FISC.COMERCIO S/ARANC 21,00%",Iva 21,RETENCION IVA,PER B.A.I.BR.DN.01/04,RETENCION IMP.GANANCIAS,RETENCION ING.BRUTOS,RETENCION ING.BRUTOS SIRTAC,PERC.I.B. CORRIENTES R.165/00,"PERCEPCION IVA R.G. 2408 3,00 %",TOTAL DEDUCCIONES,TOTAL LIQUIDACION,Diferencia,Local,Banco
0,Visa Debito,25098024,149309,13685857.00,109483.761905,22991.59,4.62,67310.35,0.00,67310.35,492570.26,109486.84,0.00,3258.06,872411.14,12813445.86,0.071905,ROSARIO,Banco Frances
1,Visa Debito,25107096,149310,4154368.81,33234.238095,6979.19,4.41,12589.49,232.63,12589.49,81919.11,33234.93,0.00,507.64,181286.70,3973082.11,0.018095,9 DE JULIO,Banco Frances
2,Visa Debito,25107105,149311,10365430.87,82921.333333,17413.48,4.41,51412.54,580.45,51412.54,203593.69,82923.44,0.00,2430.63,492688.10,9872742.77,0.003333,BAHIA I,Banco Frances
3,Visa Debito,25107118,149312,9265906.03,74125.380952,15566.33,4.41,44226.70,518.87,44226.70,182066.53,74127.27,0.00,2087.93,436945.69,8828960.34,0.020952,BAHIA II,Banco Frances
4,Visa Debito,25107120,149313,9395495.20,75162.428571,15784.11,4.41,46601.67,0.00,46601.67,0.00,75163.94,0.00,2045.21,261359.06,9134136.14,-0.031429,BARILOCHE,Banco Frances
5,Visa Debito,25107146,149314,4434861.19,35478.333333,7450.45,4.41,12369.42,248.36,12369.42,87492.89,35478.91,0.00,456.08,191343.71,4243517.48,0.153333,BRAGADO I,Banco Frances
6,Visa Debito,25107159,149315,4269617.44,34156.047619,7172.77,4.41,13697.66,239.05,13697.66,84161.32,34156.93,0.00,568.51,187850.02,4081767.42,-0.072381,TRENQUE LAUQUEN,Banco Frances
7,Visa Debito,25107161,149316,25132593.94,201055.190476,42221.59,4.41,124657.69,0.00,124657.69,617055.59,201060.76,0.00,6033.39,1316741.79,23815852.15,0.110476,CABILDO,Banco Frances
8,Visa Debito,25107174,149317,13813276.26,110502.714286,23205.57,4.41,68513.88,0.00,68513.88,339143.65,110506.23,0.00,3316.34,723702.28,13089573.98,-0.015714,STA FE CAPITAL,Banco Frances
9,Visa Debito,25107187,149318,19418918.98,155346.523810,32622.77,4.41,96317.84,0.00,96317.84,476773.39,155351.35,0.00,4662.10,1017391.82,18401527.16,-0.006190,STA FE Y ANASAG,Banco Frances


In [24]:
writer = ExcelWriter(r'C:\Users\Juan M Cabral\Desktop\Liquidaciones tarjetas\11-22\Visa Debito 11-22.xlsx')
tarjeta_visa_debito.to_excel(writer, 'Hoja de datos', index=False)
writer.save()

C:\Users\Juan M Cabral\AppData\Local\Temp\ipykernel_7468\1145517172.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [11]:
tarjeta_visa_debito['Diferencia'].unique(order=False)

TypeError: Series.unique() got an unexpected keyword argument 'order'

In [12]:
a = tarjeta_visa_debito['Diferencia'].unique()
print(sorted(a))

[-0.02333333333444898, -0.023333333333312112, -0.02333333333262999, -0.023333333330811, -0.022857142857446888, -0.02285714285699214, -0.022857142856082646, -0.02238095238135429, -0.02238095238101323, -0.022380952380899544, -0.022380952380444796, -0.0223809523795353, -0.021904761906625936, -0.021904761905261694, -0.021904761904806946, -0.02190476190389745, -0.021904761902987957, -0.021428571429169097, -0.02142857142871435, -0.021428571428600662, -0.021428571428259602, -0.020952380955350236, -0.020952380953531247, -0.020952380952621752, -0.020952380952394378, -0.020952380952167005, -0.020952380951712257, -0.02047619048244087, -0.02047619047880289, -0.020476190476983902, -0.020476190476074407, -0.020476190475164913, -0.020476190471526934, -0.020000000004074536, -0.020000000002255547, -0.020000000000436557, -0.01999999999998181, -0.019999999998617568, -0.01999999999679858, -0.01999999998952262, -0.019523809525708202, -0.019523809523889213, -0.019523809523775526, -0.01952380952366184, -0.01